In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from psycopg2.extras import execute_batch

In [ ]:
#Связь с БД
get_db_connection():
    return psycopg2.connect(**DB_CONFIG)

In [ ]:
#Конфигурация
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'  
LIMIT = 100  # Устнавливаем лимит отзывов для обработки, чтобы не груззить базу

In [ ]:
def generate_limited_embeddings():
    model = SentenceTransformer(MODEL_NAME)
    
    #Получаем отзывы для обработки
    try:
        with get_db_connection() as conn:
            with conn.cursor() as cursor:
                cursor.execute("""
                    SELECT r.review_id, r.text 
                    FROM reviews r
                    WHERE NOT EXISTS(
                        SELECT 1 FROM embeddings e
                        WHERE e.review_id = r.review_id
                    )
                    ORDER BY r.review_id
                    LIMIT %s
                """, (LIMIT,))
                
                batch = cursor.fetchall()
                if not batch:
                    print("Нет новых отзывов для обработки")
                    return 0

                review_ids, texts = zip(*batch)
                print(f"Начало обработки {len(batch)} отзывов...")

                #Генерация эмбеддингов
                embeddings = model.encode(texts, convert_to_tensor=False) 

                #Вставляем с использованием execute_batch для большей эффективности
                try:
                    execute_batch(
                        cursor,
                        "INSERT INTO embeddings (review_id, vector) VALUES (%s, %s)",
                        [(rid, emb.tolist()) for rid, emb in zip(review_ids, embeddings)],
                        page_size=100
                    )
                    conn.commit()
                    print(f"Успешно обработано {len(batch)} отзывов")
                    return len(batch)
                    
                except Exception as e:
                    conn.rollback()
                    print(f"Ошибка при вставке в базу данных: {str(e)}")
                    raise

    except psycopg2.Error as e:
        print(f"Ошибка подключения к базе данных: {str(e)}")
        return 0
if __name__ == "__main__":
    generate_limited_embeddings()